## Python Database Connection

### Create Table covid_deaths in Covid19 Database

In [10]:
#<--import necessary libraries-->
import pandas as pd
import psycopg2

In [9]:
#<-- import the dataset -->
data = pd.read_csv('covid_deaths_data.csv')
data.head(10)

,iso_code,continent,location,date,population,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,03-01-2020,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,04-01-2020,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,05-01-2020,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,06-01-2020,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,07-01-2020,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AFG,Asia,Afghanistan,08-01-2020,41128772,NaN,0.0,0.0,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AFG,Asia,Afghanistan,09-01-2020,41128772,NaN,0.0,0.0,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AFG,Asia,Afghanistan,10-01-2020,41128772,NaN,0.0,0.0,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AFG,Asia,Afghanistan,11-01-2020,41128772,NaN,0.0,0.0,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AFG,Asia,Afghanistan,12-01-2020,41128772,NaN,0.0,0.0,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302732 entries, 0 to 302731
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   iso_code                            302732 non-null  object 
 1   continent                           288330 non-null  object 
 2   location                            302732 non-null  object 
 3   date                                302732 non-null  object 
 4   population                          302732 non-null  int64  
 5   total_cases                         266771 non-null  float64
 6   new_cases                           294064 non-null  float64
 7   new_cases_smoothed                  292800 non-null  float64
 8   total_deaths                        246214 non-null  float64
 9   new_deaths                          294139 non-null  float64
 10  new_deaths_smoothed                 292909 non-null  float64
 11  total_cases_per_million   

In [47]:
#<--view th col names -->
col = data.columns
col

Index(['iso_code', 'continent', 'location', 'date', 'population',
       'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths',
       'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million'],
      dtype='object')

In [48]:
#<--find the length of the col, to find the number of columns -->
len(col)

26

In [59]:
#<--create table for sql and choose the data types appropriately-->
create_table = '''create table covid_deaths('''
for item in col:
    if(item in ['iso_code', 'continent', 'location']):
        create_table+=item+' '+'text'+','
    elif(item=='date'):
        create_table+=item+' '+'date'+','
    elif(item in ['population','total_cases', 'new_cases','total_deaths',
                  'new_deaths','icu_patients','hosp_patients',
                  'weekly_icu_admissions','weekly_hosp_admissions']):
        create_table+=item+' '+'bigint'+','
    elif(item == 'weekly_hosp_admissions_per_million'):
        create_table+=item+' '+'numeric(20,5)'+')'
    else:
        create_table+=item+' '+'numeric(20,5)'+','
print(create_table) 

create table covid_deaths(iso_code text,continent text,location text,date date,population bigint,total_cases bigint,new_cases bigint,new_cases_smoothed numeric(20,5),total_deaths bigint,new_deaths bigint,new_deaths_smoothed numeric(20,5),total_cases_per_million numeric(20,5),new_cases_per_million numeric(20,5),new_cases_smoothed_per_million numeric(20,5),total_deaths_per_million numeric(20,5),new_deaths_per_million numeric(20,5),new_deaths_smoothed_per_million numeric(20,5),reproduction_rate numeric(20,5),icu_patients bigint,icu_patients_per_million numeric(20,5),hosp_patients bigint,hosp_patients_per_million numeric(20,5),weekly_icu_admissions bigint,weekly_icu_admissions_per_million numeric(20,5),weekly_hosp_admissions bigint,weekly_hosp_admissions_per_million numeric(20,5))


In [60]:
#<--connect to database and create the table-->
try:
    conn = psycopg2.connect(database = 'Covid19',
                           user = 'postgres',
                           password = 't***********$',
                           host= 'localhost',
                           port = '5432')

    cursor = conn.cursor()

    # ececute the command to create a table
    cursor.execute(create_table)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()

In [61]:
#<-- create a function to insert records to the table created -->
def insertrecord(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z):
    connection = None
    try:
        connection = psycopg2.connect(database = 'Covid19',
                                      user = "postgres",
                                      password = "t***********$",
                                      host = "localhost",
                                      port = "5432")
        cursor= connection.cursor()
        cursor.execute('''insert into covid_deaths
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''',(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z))
        connection.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if connection is not None:
            cursor.close()
            connection.close()

In [62]:
#<-- find the column types from the original data and group them accordingly-->
l = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
d ={}
for i in range(len(l)):
    if l[i] not in d:
        d[i]=col[i]
print(d)

{0: 'iso_code', 1: 'continent', 2: 'location', 3: 'date', 4: 'population', 5: 'total_cases', 6: 'new_cases', 7: 'new_cases_smoothed', 8: 'total_deaths', 9: 'new_deaths', 10: 'new_deaths_smoothed', 11: 'total_cases_per_million', 12: 'new_cases_per_million', 13: 'new_cases_smoothed_per_million', 14: 'total_deaths_per_million', 15: 'new_deaths_per_million', 16: 'new_deaths_smoothed_per_million', 17: 'reproduction_rate', 18: 'icu_patients', 19: 'icu_patients_per_million', 20: 'hosp_patients', 21: 'hosp_patients_per_million', 22: 'weekly_icu_admissions', 23: 'weekly_icu_admissions_per_million', 24: 'weekly_hosp_admissions', 25: 'weekly_hosp_admissions_per_million'}


**NOTE**
1. (0,1,2) - text
2. 3 - date
3. (4,5,6,8,9,18,20,22,24) - integer
4. rest - numeric(8,4)

In [63]:
#<-- call the insertrecord funtion and insert data to the table -->
f = open('covid_deaths_data.csv','r')
f.readline()
for line in f:
    l_insert =[]
    line = line.strip('\n').split(',')
    for i in range(len(line)):
        if i in [0,1,2,3]:
            if line[i]=='':
                l_insert.append(None)
            else:
                l_insert.append(line[i])
        elif i in [4,5,6,8,9,18,20,22,24]:
            if line[i]=='':
                l_insert.append(None)
            else:
                l_insert.append(int(line[i]))
        else:
            if line[i]=='':
                l_insert.append(None)
            else:
                l_insert.append(float(line[i]))
    insertrecord(l_insert[0],l_insert[1],l_insert[2],l_insert[3],l_insert[4],
                l_insert[5],l_insert[6],l_insert[7],l_insert[8],l_insert[9],
                l_insert[10],l_insert[11],l_insert[12],l_insert[13],l_insert[14],
                l_insert[15],l_insert[16],l_insert[17],l_insert[18],l_insert[19],
                l_insert[20],l_insert[21],l_insert[22],l_insert[23],l_insert[24],
                l_insert[25])

### Create Table covid_vaccinations in Covid19 Database

In [2]:
#<--import the next dataset -->
data1 = pd.read_csv('covid_vaccinations_data.csv')
data1.head(10)

,iso_code,continent,location,date,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
5,AFG,Asia,Afghanistan,2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
6,AFG,Asia,Afghanistan,2020-01-09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
7,AFG,Asia,Afghanistan,2020-01-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
8,AFG,Asia,Afghanistan,2020-01-11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
9,AFG,Asia,Afghanistan,2020-01-12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [3]:
col = data1.columns
col

Index(['iso_code', 'continent', 'location', 'date', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed',
       'new_people_vaccinated_smoothed_per_hundred', 'stringency_index',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'handwashing_facilities', 'hospital_beds_per_thousand',
  

In [4]:
#<--find the number of columns in the dataset -->
len(col)

45

In [5]:
#<--create a second table query -->
create_table = '''create table covid_vaccinations('''
for item in col:
    if(item in ['iso_code', 'continent', 'location','tests_units']):
        create_table+=item+' '+'text'+','
    elif(item=='date'):
        create_table+=item+' '+'date'+','
    elif(item in ['total_tests', 'new_tests','total_vaccinations',
                  'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
                  'new_vaccinations',]):
        create_table+=item+' '+'bigint'+','
    elif(item == 'excess_mortality_cumulative_per_million'):
        create_table+=item+' '+'numeric(20,5)'+')'
    else:
        create_table+=item+' '+'numeric(20,5)'+','
print(create_table)

create table covid_vaccinations(iso_code text,continent text,location text,date date,total_tests bigint,new_tests bigint,total_tests_per_thousand numeric(20,5),new_tests_per_thousand numeric(20,5),new_tests_smoothed numeric(20,5),new_tests_smoothed_per_thousand numeric(20,5),positive_rate numeric(20,5),tests_per_case numeric(20,5),tests_units text,total_vaccinations bigint,people_vaccinated bigint,people_fully_vaccinated bigint,total_boosters bigint,new_vaccinations bigint,new_vaccinations_smoothed numeric(20,5),total_vaccinations_per_hundred numeric(20,5),people_vaccinated_per_hundred numeric(20,5),people_fully_vaccinated_per_hundred numeric(20,5),total_boosters_per_hundred numeric(20,5),new_vaccinations_smoothed_per_million numeric(20,5),new_people_vaccinated_smoothed numeric(20,5),new_people_vaccinated_smoothed_per_hundred numeric(20,5),stringency_index numeric(20,5),population_density numeric(20,5),median_age numeric(20,5),aged_65_older numeric(20,5),aged_70_older numeric(20,5),gdp

In [6]:
#<--connect to database and create table covis_vaccinations in Covid19 database-->
try:
    conn = psycopg2.connect(database = 'Covid19',
                           user = 'postgres',
                           password = 't***********$',
                           host= 'localhost',
                           port = '5432')

    cursor = conn.cursor()

    # ececute the command to create a table
    cursor.execute(create_table)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()

In [7]:
#<--function to insert record to the table created in the above cell-->
def insertrecord(a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, 
                 a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, 
                 a26, a27, a28, a29, a30, a31, a32, a33, a34, a35, a36, 
                 a37, a38, a39, a40, a41, a42, a43, a44):
    connection = None
    try:
        connection = psycopg2.connect(database = 'Covid19',
                                      user = "postgres",
                                      password = "t***********$",
                                      host = "localhost",
                                      port = "5432")
        cursor= connection.cursor()
        cursor.execute('''insert into covid_vaccinations
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''',
                       (a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, 
                        a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, 
                        a26, a27, a28, a29, a30, a31, a32, a33, a34, a35, a36, 
                        a37, a38, a39, a40, a41, a42, a43, a44))
        connection.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if connection is not None:
            cursor.close()
            connection.close()

In [8]:
#<--number the columns to create groups of columns of same data type-->
l = [i for i in range(45)]
d={}
for i in range(45):
    d[l[i]]=col[i]
print(d)

{0: 'iso_code', 1: 'continent', 2: 'location', 3: 'date', 4: 'total_tests', 5: 'new_tests', 6: 'total_tests_per_thousand', 7: 'new_tests_per_thousand', 8: 'new_tests_smoothed', 9: 'new_tests_smoothed_per_thousand', 10: 'positive_rate', 11: 'tests_per_case', 12: 'tests_units', 13: 'total_vaccinations', 14: 'people_vaccinated', 15: 'people_fully_vaccinated', 16: 'total_boosters', 17: 'new_vaccinations', 18: 'new_vaccinations_smoothed', 19: 'total_vaccinations_per_hundred', 20: 'people_vaccinated_per_hundred', 21: 'people_fully_vaccinated_per_hundred', 22: 'total_boosters_per_hundred', 23: 'new_vaccinations_smoothed_per_million', 24: 'new_people_vaccinated_smoothed', 25: 'new_people_vaccinated_smoothed_per_hundred', 26: 'stringency_index', 27: 'population_density', 28: 'median_age', 29: 'aged_65_older', 30: 'aged_70_older', 31: 'gdp_per_capita', 32: 'extreme_poverty', 33: 'cardiovasc_death_rate', 34: 'diabetes_prevalence', 35: 'female_smokers', 36: 'male_smokers', 37: 'handwashing_facilit

1. (0,1,2,12) - string
2. 3 - date(leave as string)
3. (4,5,13,14,15,16,17) - integers
4. Rest - float

In [9]:
#,--call the function insertrecord to insert records to the table created above -->
f = open('covid_vaccinations_data.csv','r')
f.readline()
for line in f:
    l_insert =[]
    line = line.strip('\n').split(',')
    for i in range(len(line)):
        if i in [0,1,2,3,12]:
            if line[i]=='':
                l_insert.append(None)
            else:
                l_insert.append(line[i])
        elif i in [4,5,13,14,15,16,17]:
            if line[i]=='':
                l_insert.append(None)
            else:
                l_insert.append(int(line[i]))
        else:
            if line[i]=='':
                l_insert.append(None)
            else:
                l_insert.append(float(line[i]))
    insertrecord(l_insert[0],l_insert[1],l_insert[2],l_insert[3],l_insert[4],
                l_insert[5],l_insert[6],l_insert[7],l_insert[8],l_insert[9],
                l_insert[10],l_insert[11],l_insert[12],l_insert[13],l_insert[14],
                l_insert[15],l_insert[16],l_insert[17],l_insert[18],l_insert[19],
                l_insert[20],l_insert[21],l_insert[22],l_insert[23],l_insert[24],
                l_insert[25],l_insert[26],l_insert[27],l_insert[28],l_insert[29],
                l_insert[30],l_insert[31],l_insert[32],l_insert[33],l_insert[34],
                l_insert[35],l_insert[36],l_insert[37],l_insert[38],l_insert[39],
                l_insert[40],l_insert[41],l_insert[42],l_insert[43],l_insert[44])

### Database Covid19 created successfully! Let's explore with SQL!